In [ ]:
import kagglehub
import pandas as pd

# Descarga el dataset (esto devuelve la ruta de la carpeta)
path = kagglehub.dataset_download("mashlyn/online-retail-ii-uci")

import os
print(os.listdir(path))

file_path = f"{path}/online_retail_II.csv"
df = pd.read_csv(file_path)

print(df.info())

Using Colab cache for faster access to the 'online-retail-ii-uci' dataset.
['online_retail_II.csv']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB
None


In [ ]:
df['TotalRevenue'] = df['Quantity']*df['Price']

In [ ]:
df['IsCancellation'] = df['Invoice'].str.startswith('C', na=False)

In [ ]:
# Limpiar datos: Quitar cantidades negativas que no son cancelaciones y precios cero
df_clean = df[(df['Quantity'] > 0) & (df['Price']>0)].copy()

In [ ]:
# Formatear fechas para análisis temporal
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])
df_clean['MonthYear']= df_clean['InvoiceDate'].dt.to_period('M')

print(f"Registros originales: {len(df)}")
print(f"Registros limpios: {len(df_clean)}")

Registros originales: 1067371
Registros limpios: 1041671


In [ ]:
df_clean['MonthYear'] = df_clean['MonthYear'].astype(str)
import sqlite3
#conexion a db local
conexion = sqlite3.connect('retail_database.db')
#carga del dataframe
df_clean.to_sql('sales',conexion, if_exists='replace', index='False')

#consultar los top clientes que mas gastaron por pais
query ="""
SELECT
    Country,
    "Customer ID",
    SUM(TotalRevenue) AS TotalSpent
FROM sales
GROUP BY Country,"Customer ID"
ORDER BY TotalSpent DESC
LIMIT 10;
"""
top_customers = pd.read_sql(query, conexion)
print(top_customers)



          Country  Customer ID    TotalSpent
0  United Kingdom          NaN  3.148204e+06
1  United Kingdom      18102.0  6.088217e+05
2     Netherlands      14646.0  5.286025e+05
3            EIRE      14156.0  3.139464e+05
4            EIRE      14911.0  2.959726e+05
5  United Kingdom      17450.0  2.469731e+05
6  United Kingdom      13694.0  1.964828e+05
7  United Kingdom      17511.0  1.756036e+05
8  United Kingdom      16446.0  1.684725e+05
9  United Kingdom      16684.0  1.471428e+05


In [ ]:
snapshot_date = df_clean['InvoiceDate'].max() + pd.Timedelta(days=1)
rfm= df_clean.groupby('Customer ID').agg({
    'InvoiceDate' : lambda x: (snapshot_date - x.max()).days,
    'Invoice': 'nunique',
    'TotalRevenue': 'sum'
})

rfm.columns = ['Recency', 'Frequency', 'Monetary']
# Dividir en grupos del 1 al 5 (Segmentación)
rfm['R_Score'] = pd.qcut(rfm['Recency'],5, labels=[5,4,3,2,1])
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'),5, labels=[1,2,3,4,5])
rfm['M_Score'] = pd.qcut(rfm['Monetary'],5, labels=[1,2,3,4,5])

rfm['RFM_Segment'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str)

In [ ]:
import pandas as pd
import sqlite3
import plotly.express as px
import numpy as np
from sklearn.linear_model import LinearRegression

# Conexión a la base de datos que ya creamos
conn = sqlite3.connect('retail_database.db')
df = pd.read_sql("SELECT * FROM sales", conn)
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Función para obtener predicción por país
def obtener_prediccion(nombre_pais):
    df_p = df[df['Country'] == nombre_pais]
    # Agrupar por mes
    monthly = df_p.groupby('MonthYear')['TotalRevenue'].sum().reset_index()
    monthly['Month_Ordinal'] = np.arange(len(monthly))

    # Entrenar modelo
    X = monthly[['Month_Ordinal']]
    y = monthly['TotalRevenue']
    model = LinearRegression().fit(X, y)

    # Predecir próximo mes
    pred = model.predict([[len(monthly)]])[0]
    return monthly, pred

# Ejemplo con un país
pais_demo = 'United Kingdom'
datos_mes, proxima_venta = obtener_prediccion(pais_demo)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [ ]:
from IPython.display import display, HTML

# 1. Gráfico de Predicción
fig_pred = px.scatter(datos_mes, x='Month_Ordinal', y='TotalRevenue',
                 trendline="ols", title=f"Predicción de Ventas en {pais_demo}")

# 2. Gráfico de Top Productos
top_p = df[df['Country'] == pais_demo].groupby('Description')['Quantity'].sum().nlargest(10).reset_index()
fig_bar = px.bar(top_p, x='Quantity', y='Description', orientation='h', title="Top 10 Productos")

# Mostrar métricas y gráficas
print(f"--- ANÁLISIS PARA {pais_demo} ---")
print(f"Venta estimada próximo mes: £{proxima_venta:,.2f}")

fig_pred.show()
fig_bar.show()

--- ANÁLISIS PARA United Kingdom ---
Venta estimada próximo mes: £827,467.88


In [47]:
readme_content = """
# 🛒 Online Retail Intelligence & Predictive Analytics
![Python](https://img.shields.io/badge/Python-3776AB?style=for-the-badge&logo=python&logoColor=white)
![SQL](https://img.shields.io/badge/SQL-003B57?style=for-the-badge&logo=sqlite&logoColor=white)
![Machine Learning](https://img.shields.io/badge/Scikit--Learn-F7931E?style=for-the-badge&logo=scikit-learn&logoColor=white)

## 📋 Descripción del Proyecto
Este proyecto desarrolla un ecosistema de datos completo para un negocio de Retail, transformando más de **500,000 transacciones** en insights estratégicos. Se abarca desde la ingesta de datos masivos hasta la predicción de ventas utilizando Machine Learning.

## 🚀 Características Principales
* **ETL Pipeline:** Limpieza y normalización de datos reales (Kaggle Dataset) manejando cancelaciones, valores nulos y tipos de datos complejos.
* **SQL Warehouse:** Implementación de una base de datos relacional en SQLite para consultas de negocio rápidas.
* **Forecasting de Ventas:** Modelo de **Regresión Lineal** que predice el volumen de ventas del próximo mes con base en tendencias históricas.
* **Análisis RFM:** Segmentación avanzada de clientes (Recencia, Frecuencia, Valor Monetario) para estrategias de marketing dirigidas.

## 🛠️ Stack Tecnológico
- **Lenguaje:** Python (Pandas, Numpy)
- **Base de Datos:** SQLite / SQL
- **Machine Learning:** Scikit-Learn (Linear Regression)
- **Visualización:** Plotly & Matplotlib
- **Big Data (Bonus):** Procesamiento inicial con Hadoop MapReduce y Pig.

## 📊 Resultados Destacados
1. **Predicción Precisa:** El sistema genera estimaciones de ingresos por país, permitiendo una optimización del stock.
2. **Top Productos:** Identificación automática de los 10 productos con mayor rotación.
3. **Tendencias Temporales:** Análisis de estacionalidad que revela picos de venta en periodos específicos.

## 📁 Estructura del Repositorio
- `notebook_analisis.ipynb`: Flujo completo de limpieza, SQL y ML.
- `app.py`: Código del dashboard interactivo.
- `retail_database.db`: Base de datos procesada.

---
Creado con ❤️ para mi portafolio de Ciencia de Datos.
Con ayuda de IA(gimini)
"""

with open("README1.md", "w") as f:
    f.write(readme_content)

print("✅ Archivo README1.md generado con éxito.")

✅ Archivo README1.md generado con éxito.
